In [1]:
import pandas as pd
import statsmodels.api as sm

Warning: Runnig this model gives out warnings that we could not deal with. It may has to do with memory, or the form and shape of our current data. It will cause some tables to be filled with missing values each time you run the model. It seemed that the more you run it, the more missing values it creates.
Same for the second model in the next block.

In [ ]:
df_markets = pd.read_csv('../../01_data/02_pre/markets_info.csv')


with open('../../03_documentation/multinomial_reg_results.txt', 'w') as f:
    for ticker in df_markets['ticker']:
        df= pd.read_csv(f'../../01_data/03_analysis/{ticker}.csv')
        
        #Dealing with missing values
        df[['sum', 'max_mag', 'max_sig', 'min_depth', 'min_dist', 'tsunami']] = df[['sum', 'max_mag', 'max_sig', 'min_depth', 'min_dist', 'tsunami']].fillna(0)
        df['rev_dist'] = df['min_dist'].apply(lambda x: 0 if x == 0 else 1 / x)
        df['rev_dist_sq'] = df['rev_dist'] * df['rev_dist']
        df['rev_depth'] = df['min_depth'].apply(lambda x: 0 if x == 0 else 1 / x)
    
        #This part creates the regression model
        X = sm.add_constant(df[['num', 'max_sig', 'rev_dist']])
        y = df['effect']
        mlogit_model = sm.MNLogit(y, X)
        mlogit_model.fit(method='bfgs', maxiter=200)
        result = mlogit_model.fit()
        

        #Saving it to the file
        f.write(f"Ticker: {ticker}\n")
        f.write(result.summary().as_text())
        f.write("\n" + "="*80 + "\n")

As the model assumes that depth and distance are reversely associated with change in the market, when we assign 0 to them the model will be confused. Therefore I assigned maximum, unreal numbers, to both to see if they might matter. Turns out, they don't.

In [ ]:
df_markets = pd.read_csv('data/test/markets.csv')


with open('data/test/linear/results_logistic corrected distance and depth.txt', 'w') as f:
    for ticker in df_markets['Ticker']:
        df= pd.read_csv(f'data/test/linear/{ticker}.csv')
        
        #Dealing with missing values
        df[['sum', 'max_mag', 'max_sig', 'tsunami']] = df[['sum', 'max_mag', 'max_sig', 'tsunami']].fillna(0)
        df[['min_dist']] = df[['min_dist']].fillna(20000)
        df[['min_depth']] = df[['min_depth']].fillna(20000)
        df['rev_dist'] = df['min_dist'].apply(lambda x: 0 if x == 0 else 1 / x)
        df['rev_dist_sq'] = df['rev_dist'] * df['rev_dist']
        df['rev_depth'] = df['min_depth'].apply(lambda x: 0 if x == 0 else 1 / x)
    
    
        #This part creates the regression model
        X = sm.add_constant(df[['num', 'max_sig', 'rev_dist']])
        y = df['effect']
        mlogit_model = sm.MNLogit(y, X)
        mlogit_model.fit(method='bfgs', maxiter=200)
        result = mlogit_model.fit()
        

        #Saving it to the file
        f.write(f"Ticker: {ticker}\n")
        f.write(result.summary().as_text())
        f.write("\n" + "="*80 + "\n")

c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3059: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3060: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3027: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3028: RuntimeWarning: invalid value encountered in divide
  return eXB/eXB.sum(1)[:,None]
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3027: RuntimeWarning: overflow encountered in 

         Current function value: nan
         Iterations: 3
         Function evaluations: 125
         Gradient evaluations: 125
         Current function value: 0.468043
         Iterations: 35


c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3059: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3060: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3027: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3028: RuntimeWarning: invalid value encountered in divide
  return eXB/eXB.sum(1)[:,None]
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3059: RuntimeWarning: divide by zero encounter

Optimization terminated successfully.
         Current function value: 0.403868
         Iterations: 32
         Function evaluations: 39
         Gradient evaluations: 39
Optimization terminated successfully.
         Current function value: nan
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.352560
         Iterations: 34
         Function evaluations: 43
         Gradient evaluations: 43
         Current function value: 0.949703
         Iterations: 35


c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3059: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3060: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3027: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3028: RuntimeWarning: invalid value encountered in divide
  return eXB/eXB.sum(1)[:,None]
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization fa

...

c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3059: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3060: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3027: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3028: RuntimeWarning: invalid value encountered in divide
  return eXB/eXB.sum(1)[:,None]


Optimization terminated successfully.
         Current function value: 0.583107
         Iterations: 99
         Function evaluations: 107
         Gradient evaluations: 107
Optimization terminated successfully.
         Current function value: nan
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.363300
         Iterations: 34
         Function evaluations: 41
         Gradient evaluations: 41
         Current function value: 1.251875
         Iterations: 35


c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3059: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3060: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3059: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\disc

Optimization terminated successfully.
         Current function value: 0.175238
         Iterations: 35
         Function evaluations: 46
         Gradient evaluations: 46
Optimization terminated successfully.
         Current function value: nan
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.158331
         Iterations: 23
         Function evaluations: 29
         Gradient evaluations: 29
Optimization terminated successfully.
         Current function value: nan
         Iterations 9


c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3059: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3060: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3027: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
c:\Users\SJANO\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\discrete\discrete_model.py:3028: RuntimeWarning: invalid value encountered in divide
  return eXB/eXB.sum(1)[:,None]
